In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("/Users/tuhinalambijoy/Downloads/final_data.csv")
df

,age,annual_income,financial_responsibility,on-time_payment,credit_utilization,credit_score
0,56,780.24,6,68.11,50.16,646
1,46,1657.77,2,86.29,9.86,850
2,32,658.35,7,94.72,68.35,656
3,60,1426.70,3,76.01,37.54,850
4,25,1568.41,3,76.01,12.64,850
...,...,...,...,...,...,...
995,22,978.50,6,95.78,33.06,789
996,40,761.27,1,80.82,74.47,580
997,27,1111.06,8,62.31,41.24,773
998,61,991.12,7,84.53,19.58,816


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [4]:
X = df[["annual_income"]].values
y = df["credit_score"].values

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [5]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [6]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)
y_pred_lr = np.clip(y_pred_lr, 300, 850)  # enforce max credit score

print("Linear Regression RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_lr)))
print("Linear Regression R²:", r2_score(y_test, y_pred_lr))
print("Coefficient:", lr_model.coef_[0])
print("Intercept:", lr_model.intercept_)


Linear Regression RMSE: 53.27351278485576
Linear Regression R²: 0.825437685650147
Coefficient: 0.21241526933665544
Intercept: 497.80541105313944


In [7]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
y_pred_rf = np.clip(y_pred_rf, 300, 850)

print("Random Forest RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("Random Forest R²:", r2_score(y_test, y_pred_rf))


Random Forest RMSE: 53.04890176327618
Random Forest R²: 0.826906556850847


In [8]:
ann_model = Sequential([
    Dense(16, activation='relu', input_shape=(1,)),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')  # linear output
])

ann_model.compile(optimizer='adam', loss='mse')
ann_model.fit(X_train_scaled, y_train, epochs=100, batch_size=2, verbose=0)

y_pred_ann = ann_model.predict(X_test_scaled)
y_pred_ann = np.clip(y_pred_ann, 300, 850)

print("ANN RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_ann)))
print("ANN R²:", r2_score(y_test, y_pred_ann))


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
ANN RMSE: 53.33882519738788
ANN R²: 0.8250094056129456


In [10]:
# Example new incomes
new_incomes = np.array([[980], [750], [850]])
new_scaled = scaler.transform(new_incomes)

# Linear Regression
pred_lr = np.clip(lr_model.predict(new_incomes), 300, 850)

# Random Forest
pred_rf = np.clip(rf_model.predict(new_incomes), 300, 850)

# ANN
pred_ann = np.clip(ann_model.predict(new_scaled), 300, 850)

for i, inc in enumerate(new_incomes.flatten()):
    print(f"Income: {inc} → LR: {pred_lr[i]:.1f}, RF: {pred_rf[i]:.1f}, ANN: {pred_ann[i][0]:.1f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Income: 980 → LR: 706.0, RF: 713.5, ANN: 705.5
Income: 750 → LR: 657.1, RF: 664.0, ANN: 656.7
Income: 850 → LR: 678.4, RF: 674.1, ANN: 677.9
